In [3]:
import pandas as pd
import numpy as np
import psycopg2
import boto3
import configparser

## Configuración de la base de datos

In [4]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

## Conexión a base de datos

In [5]:
import sql_queries

try:
    db_conn = psycopg2.connect(
        database = config.get('RDS', 'DB_NAME'),
        user = config.get('RDS', 'DB_USER'),
        password = config.get('RDS', 'DB_PASSWORD'),
        host = config.get('RDS', 'DB_HOST'),
        port = config.get('RDS', 'DB_PORT')
    )
    cursor = db_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de datos creada exitosamente.")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)


Base de datos creada exitosamente.
